In [63]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf

In [64]:
raw_data = pd.read_csv('HR_comma_sep.csv')

In [65]:
salary_mapping = {'low':1,'medium':3,'high':5}
raw_data['salary'] = raw_data.salary.replace(salary_mapping)

In [66]:
dummies = pd.get_dummies(raw_data['sales'], prefix='sales', drop_first=False)
raw_data = pd.concat([raw_data, dummies], axis=1)
raw_data = raw_data.drop('sales',axis=1)

In [67]:
columns_to_be_normalized = ['number_project','average_montly_hours','time_spend_company','salary']
scaled_features = {}
for each in columns_to_be_normalized:
    mean, std = raw_data[each].mean(), raw_data[each].std()
    scaled_features[each] = [mean, std]
    raw_data.loc[:, each] = (raw_data[each] - mean)/std

In [68]:
number_of_columns = raw_data.columns.size
number_of_rows = len(raw_data)
test_size = 1000
validation_size = 3000
target_field = ['left']

In [69]:
raw_data = shuffle(raw_data)

In [70]:
test_data = raw_data[:test_size]
validation_data = raw_data[test_size:test_size+validation_size]
train_data = raw_data[test_size+validation_size:len(raw_data)]

In [71]:
train_features,train_target = train_data.drop(target_field,axis=1),train_data[target_field]
validation_features,validation_target = validation_data.drop(target_field,axis=1),validation_data[target_field]
test_features,test_target = test_data.drop(target_field,axis=1),test_data[target_field]

In [91]:
# Parameters
learning_rate = 0.1
training_epochs = 2000
display_step = 50
n_samples = len(train_data)

In [92]:
train_features.shape

(10999, 18)

In [93]:
x = tf.placeholder(tf.float32, [None, 18])
W = tf.Variable(tf.random_normal([18, 1])) 
B = tf.Variable(tf.zeros([1]))
y_values = tf.add(tf.matmul(x, W), B) 
y = tf.nn.softmax(y_values) 
y_ = tf.placeholder(tf.float32, [None,1])

In [ ]:
cost = tf.nn.softmax_cross_entropy_with_logits(logits=y,labels=y_)
#cost = tf.reduce_mean(tf.pow(y_ - y, 2))/(2*n_samples)
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate).minimize(cost)

In [100]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [101]:
for i in range(training_epochs):  
    sess.run(optimizer, feed_dict={x: train_features.as_matrix(), y_: train_target.as_matrix()})
    # That's all! The rest of the cell just outputs debug messages. 
    # Display logs per epoch step
    if (i) % display_step == 0:
        cc = sess.run(cost, feed_dict={x: train_features.as_matrix(), y_:train_target.as_matrix()})
        print ("Training step: ",i,"Cost: ",cc)

print ("Optimization Finished!")
training_cost = sess.run(cost, feed_dict={x: validation_features.as_matrix(), y_: validation_target.as_matrix()})
print ("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(B), '\n')    

Training step:  0 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  50 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  100 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  150 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  200 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  250 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  300 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  350 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  400 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  450 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  500 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  550 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  600 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  650 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  700 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  750 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  800 Cost:  [ 0.  0.  0. ...,  0.  0.  0.]
Training step:  8

In [102]:
output = sess.run(y, feed_dict={x: test_features })

In [103]:
correct = 0 
for i in range(len(output)):
    if(output[i] == test_target.values[i]):
        correct = correct+1
                
print(correct/len(test_target)*100)

24.8
